Explorar a base de dados utilizando recursos do PySpark.

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("InghtPlaces_Anuncios") \
    .getOrCreate()
    
dados = spark.read.json("dados/dataset_bruto.json")


In [3]:
print(f'Linhas: {dados.count()} | Colunas: {len(dados.columns)}')
print('-'*50)
# print(dados.printSchema())

Linhas: 89083 | Colunas: 3
--------------------------------------------------


In [4]:
# Para nossa análise, apenas as informações do campo "anuncio" serão relevantes. 
# Por isso, vamos focar em analisar as colunas desse campo.

dados = dados.select('anuncio.*')

dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [5]:
# O time de Data Science solicitou que fizéssemos alguns filtros nas colunas 
# tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados:

# tipo_uso: Residencial;
# tipo_unidade: Apartamento;
# tipo_anuncio: Usado.

# Para entender o impacto que esses filtros terão sob a nossa base de dados, 
# crie uma tabela de frequências para cada uma dessas colunas antes de filtrá-las.

dados.groupBy(['tipo_uso','tipo_unidade','tipo_anuncio']).count().show()


+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial|      Outros|       Usado| 7502|
|Residencial| Apartamento|  Lançamento|  235|
|Residencial| Apartamento|       Usado|66562|
|  Comercial| Apartamento|       Usado|    4|
|  Comercial|      Outros|  Lançamento|    3|
|  Comercial|      Outros|       Usado| 4443|
|  Comercial|        Casa|       Usado|   92|
|Residencial|        Casa|  Lançamento|    3|
|Residencial|      Outros|  Lançamento|   15|
|Residencial|        Casa|       Usado|10224|
+-----------+------------+------------+-----+



In [6]:
dados.count()

89083

In [7]:
# 66562 a variavel alvo solicitada pelo time de Data Science.

( 66562 / 89083 ) * 100


74.7190822042365

Realizando filtros para selecionar apenas os dados que serão utilizados.

In [8]:
# 74.8% dos dados serão utilizados para a análise.

dados = dados.filter(dados.tipo_uso == 'Residencial') \
    .filter(dados.tipo_unidade == 'Apartamento') \
    .filter(dados.tipo_anuncio == 'Usado')
    
dados.count()

66562

Remoção de Arrays as transformando em tipo Inteiro.

In [9]:
#Nesse contexto, vamos transformar os dados das colunas:

# "quartos",     |-- quartos: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "suites",      |-- suites: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "banheiros",   |-- banheiros: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "vaga",        |-- vaga: array (nullable = true)
#                | | -- element: long(containsNull=true)
# "area_total",  |-- area_total: array (nullable = true)
#                | | -- element: string(containsNull=true)
# "area_util",   |-- area_util: array (nullable = true)
#                | | -- element: string(containsNull=true)

from pyspark.sql.types import IntegerType
from pyspark.sql import functions as f

dicionario = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

for item in dicionario:
    dados = dados.withColumn(item, dados[item][0].cast("int"))


Tratamento de informações sobre localização.

In [10]:
# A coluna de endereço possui as informações de bairro e zona que devem ser extraídas para o dataset final.

dicionario = ['bairro', 'zona']

for item in dicionario:
    dados = dados.withColumn(item, dados['endereco'][item])


Transformar cada campo da coluna "valores" em uma coluna separada.

In [11]:
# Nesse contexto, transforme os dados da coluna valores em colunas separadas.

# |-- valores: array(nullable=true)
# | | -- element: struct(containsNull=true)
# | | |-- condominio: string(nullable=true)
# | | |-- iptu: string(nullable=true)
# | | |-- tipo: string(nullable=true)
# | | |-- valor: string(nullable=true)

dicionario = ['condominio', 'iptu', 'tipo', 'valor']

for item in dicionario:
    dados = dados.withColumn(item, dados['valores'][0][item])

In [12]:
#Como se trata de um estudo sobre o preço de venda dos imóveis, o time de cientistas de dados solicitou apenas as informações do tipo VENDA.
dados = dados.filter(dados.tipo == 'Venda')


In [13]:
#Remover colunas desnecessárias : Endereço, Valores e Tipo.

dados = dados.drop('endereco', 'valores', 'tipo')

In [14]:
from pyspark.sql.types import StringType

dados.printSchema()


root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



In [15]:
# dados = dados.withColumn('caracteristicas', dados['caracteristicas'].cast(StringType()))


Inicio Semana 2 - Tratamento de dados para o modelo ML.

In [16]:
dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



In [17]:
# Converter os tipos das colunas

#Podemos converter o tipo de colunas numéricas, como "andar", "banheiros", "suites" e "quartos" para o tipo inteiro. 
# Além disso também pode ser necessário converter as colunas "area_util", "condominio", "iptu" e "valor" para o tipo double.

inteiros = ['andar', 'banheiros', 'suites', 'quartos']
doubles = ['area_util', 'condominio', 'iptu', 'valor']

for item in inteiros:
    dados = dados.withColumn(item, dados[item].cast('int'))

for item in doubles:
    dados = dados.withColumn(item, dados[item].cast('double'))
    



In [18]:
dados.printSchema()

root
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



In [19]:
# A coluna caracteristicas possui listas de strings como conteúdo de suas linhas. 
# No entanto, algumas dessas listas estão sem elementos.
# Vamos transformar essas listas sem elementos em valores nulos.

# visualizando os elementos únicos das listas dessa coluna
# dados.select(f.explode("caracteristicas")).distinct().collect()

# dados = dados.withColumn("caracteristicas", f.when((f.size(f.col(
#     "caracteristicas")) == 0), f.lit(None)).otherwise(f.col('caracteristicas')))

# dados.toPandas()

# from pyspark.sql.functions import udf


# def verifica_lista_vazia(lista):
#   if len(lista) == 0:

#     return None
#   else:
#     return lista


# udf_verifica_lista_vazia = udf(verifica_lista_vazia)

# dados = dados.withColumn(
#     "caracteristicas", udf_verifica_lista_vazia(f.col("caracteristicas")))
# 415263+



In [20]:
dados.toPandas().head(5)

,andar,area_total,area_util,banheiros,caracteristicas,id,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,bairro,zona,condominio,iptu,valor
0,3,43.0,43.0,1,"[Academia, Churrasqueira, Playground, Salão de...",d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3,2,NaN,Usado,Apartamento,Residencial,1.0,Paciência,Zona Oeste,245.0,NaN,15000.0
1,2,42.0,42.0,1,"[Churrasqueira, Playground, Salão de festas, C...",085bab2c-87ad-452a-bd0f-8b1451509f84,2,NaN,Usado,Apartamento,Residencial,1.0,Paciência,Zona Oeste,0.0,0.0,15000.0
2,1,41.0,41.0,1,"[Portaria 24h, Condomínio fechado, Playground,...",18d22cbe-1b86-4764-8def-70c615f15a61,2,NaN,Usado,Apartamento,Residencial,1.0,Guaratiba,Zona Oeste,0.0,0.0,20000.0
3,3,43.0,43.0,1,"[Churrasqueira, Piscina, Playground, Salão de ...",bed8a354-9317-4426-b27e-1c798f864271,2,NaN,Usado,Apartamento,Residencial,0.0,Cosmos,Zona Oeste,285.0,NaN,20000.0
4,2,43.0,43.0,1,"[Academia, Churrasqueira, Playground, Salão de...",12a13315-d67f-48f0-9497-017b83252781,2,NaN,Usado,Apartamento,Residencial,1.0,Guaratiba,Zona Oeste,245.0,NaN,15000.0


In [35]:
#coletar apenas o texto dentro da coluna caracteristicas
lista = dados.select(f.explode("caracteristicas")).distinct().collect()

In [37]:
#coletar apenas o texto entre apass da lista
for item in lista :
    item = item[0].replace(' ', '_').lower()
    print(item)

condomínio_fechado
playground
portão_eletrônico
piscina
animais_permitidos
portaria_24h
elevador
academia
salão_de_festas
churrasqueira
